Nomes: Bel Cogo e Bruno da Siqueira Hoffmann.

Professor: Felipe de Morais.

Disciplina: Ciência de Dados e Big Data.

# Tarefa 6

Para a tarefa em questão foi utilizado o banco NoSQL MongoDB, além disso, foi usado a base de dados: https://www.kaggle.com/datasets/evgeny1928/playstation-games-info?select=games.csv. Usando o modelo `.csv` e executando os scripts abaixo para realizar a conversão do `.csv` em `.json`. A partir disso foi feita as configurações expostas na aula para execução do banco de dados Mongo e a inserção da collection.

In [3]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    # Abrir o arquivo CSV e ler o conteúdo
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = []
        
        # Converter cada linha do CSV em um dicionário e adicionar à lista
        for row in csv_reader:
            # Converter strings de listas para listas reais
            if 'platforms' in row:
                row['platforms'] = eval(row['platforms'])
            if 'genres' in row:
                row['genres'] = eval(row['genres'])
            if 'price' in row:
                row['price'] = float(row['price'])
            if 'rating' in row:
                row['rating'] = float(row['rating'])
            if 'votes' in row:
                row['votes'] = int(row['votes'])
                
            # Adicionar o dicionário à lista
            data.append(row)
    
    # Salvar a lista de dicionários no arquivo JSON
    with open(json_file_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

# Exemplo de uso:
csv_to_json('games.csv', 'games.json')


Após transformar o arquivo `.csv` em um arquivo `.json` foi necessário executar os comandos para importar o arquivo `json` para o banco de dados como uma collection, esses comando sendo executados em um terminal.

In [4]:
mongoimport --db playstation --collection games --file games.json --jsonArray

SyntaxError: invalid syntax (941529889.py, line 1)

Em seguida, foi executado o comando python abaixo para se conectar com o banco de dados.

In [36]:
import json
from pymongo import MongoClient
import pandas as pd


# se conecta com o servidor mongo da maquina
client = MongoClient('localhost', 27017)
   
# lista todos os databases
client.list_database_names()

# le a coleção denominada playstation
db = client.playstation

# desta coleção, le os arquivos de games
games = db.games

## Detalhes do Banco de Dados

O banco de dados vai possuir apenas uma collection, sendo ela a `games`. A estrutura do `document` será:
- _id: identificador default do MongoDB;
- title: título do jogo;
- platforms: lista de plataformas que o jogo executa;
- rating: avaliações do jogo, valores de 0 a 5;
- votes: quantidade de avaliações;
- developer: empresa que desenvolveu o jogo;
- release_date: data de lançamento;
- price: preço do jogo;
- genres: gêneros do jogo;
- url: url contendo com informações do jogo no site da playstation;

Um exemplo de um `document`:

```json
{
    "title": "Call of Duty®: Modern Warfare® III - Cross-Gen Bundle",
    "platforms": [
        "PS4",
        "PS5"
    ],
    "rating": 4.04,
    "votes": "334075",
    "developer": "Activision",
    "release_date": "2023-11-02",
    "price": 45.49,
    "genres": [
        "Action"
    ],
    "url": "https://store.playstation.com/en-us/concept/10001130"
}
```

## Pergunta 1

**Pergunta:** Quais são os jogos com os gêneros ação ou aventura e com a nota maior que 4.5? Indique o nome, a nota, o preço e os demais generos.

**Resposta:**

In [10]:
criteria = {
  '$or': [
    {'genres': { '$in': ['Action'] }},
    {'genres': { '$in': ['Adventure'] }}
  ],
  'rating': { '$gt': 4.5 }
}

projection = {
    'title': 1,
    'rating': 1,
    'genres': 1,
    'price': 1,
    '_id': 0
}


cursor = games.find(criteria, projection)
df = pd.DataFrame(cursor)

df

,title,rating,price,genres
0,Grand Theft Auto Online (PlayStation®5),4.52,19.99,"[Action, Adventure]"
1,Marvel’s Spider-Man 2,4.86,49.69,[Action]
2,Stellar Blade™,4.82,69.99,"[Adventure, Action]"
3,Honkai: Star Rail,4.60,0.00,"[Adventure, Role Playing Games]"
4,God of War Ragnarök,4.84,39.89,[Action]
...,...,...,...,...
1017,The Demon Rush: Legends Corrupt,5.00,17.99,"[Action, Role Playing Games]"
1018,The Journey Down: Chapter One,4.81,6.99,"[Adventure, Adventure]"
1019,The Last Door - Complete Edition,4.66,14.99,"[Adventure, Horror, Puzzle]"
1020,The Legend of Tianding,4.59,19.99,"[Adventure, Action]"


## Pergunta 2

**Pergunta:** Quais são os jogos que só dão suporte para plataforma PS5 e PS4 e que tem o preço menor que $12.00? Qual é o seu nome, preço e avaliação. Ordenando pela avaliação de maneira decrescente.

**Resposta:**

In [11]:
criteria = {"platforms.1": {"$exists": True}, "price": { "$lt": 12.00 }}

projection = {
  'title': 1,
  'rating': 1,
  'price': 1,
  '_id': 0
}

sort = {
  'rating': -1
}

cursor = games.find(criteria, projection).sort(sort)
df = pd.DataFrame(cursor)

df

,title,rating,price
0,Togges,5.00,5.99
1,Ultra Foodmess 2,5.00,4.99
2,Tales From The Under-Realm: Hazel PS4® & PS5®,5.00,11.99
3,DateJournal,5.00,10.49
4,Crazy Chicken Pirates,5.00,11.99
...,...,...,...
1495,BMX Wild Run,1.80,9.99
1496,Adventure Pinball Bundle,1.75,6.99
1497,Become the wild,1.57,8.49
1498,Book Quest PS4 & PS5,1.52,2.44


## Pergunta 3

**Pergunta:** Qual gênero do jogo mais caro? 

**Resposta:**

In [20]:
criteria = {}

projection = {
  'title': 1,
  'price': 1,
  'genres': 1,
  '_id': 0
}

sort = {
  'price': -1
}

cursor = games.find(criteria, projection).sort(sort).limit(1)
df = pd.DataFrame(cursor)

df

,title,price,genres
0,NHL® 24 X-Factor Edition PS5™ & PS4™,99.99,[Sport]


## Pergunta 4

**Pergunta:** Qual a primeira ocorrência de um registro de jogo que seja do gênero Ação (Action)?

**Resposta:**

In [48]:
criteria = {
  'genres': { '$in': ['Action'] }
}

cursor = games.find_one(criteria)
cursor


{'_id': ObjectId('67229e40524eeb899fdd2b4b'),
 'title': 'Call of Duty®: Modern Warfare® III - Cross-Gen Bundle',
 'platforms': ['PS4', 'PS5'],
 'rating': 4.04,
 'votes': 334075,
 'developer': 'Activision',
 'release_date': '2023-11-02',
 'price': 45.49,
 'genres': ['Action'],
 'url': 'https://store.playstation.com/en-us/concept/10001130'}

## Pergunta 5

**Pergunta:** Quantos jogos são exclusivos para cada uma das plataformas

**Resposta:**

In [86]:
criteriaPS4 = {
  '$and': [
    {'platforms': ['PS4']},
  ]
}
criteriaPS5 = {
  '$and': [
    {'platforms': ['PS5']},
    {'platforms.0': 'PS5'}
  ]
}

count = [{ 'Plataforma': 'PS4', 'count': games.count_documents(criteriaPS4)}, { 'Plataforma': 'PS5', 'count': games.count_documents(criteriaPS5)}]

df = pd.DataFrame(count)

df



,Plataforma,count
0,PS4,5585
1,PS5,591
